In [1]:
import numpy as np
import pandas as pd
import folium
import branca

In [4]:
df_station_coords = pd.read_csv(r"D:\capstone_F24\data\MTA_Subway_Stations_20241002.csv")
df_station_coords.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


In [3]:
df_time_series_metrics = pd.read_pickle(r"D:\capstone_F24\data\Cleaned_data\df_timeseries_metrics.pkl")
df_time_series_metrics.head()

,station_complex_id,rmse,mae,smape
0,1,384.712868,384.712868,83.853742
1,10,586.580283,586.580283,199.441926
2,100,252.846571,252.846571,124.411559
3,101,152.057256,152.057256,55.905147
4,118,81.031381,81.031381,60.615512


In [5]:
df_time_series_metrics['station_complex_id'] = df_time_series_metrics['station_complex_id'].astype(str)
df_station_coords['Complex ID'] = df_station_coords['Complex ID'].astype(str)

# Merge the dataframes
merged_df = pd.merge(
    df_time_series_metrics,
    df_station_coords[['Complex ID', 'Stop Name', 'GTFS Latitude', 'GTFS Longitude']],
    left_on='station_complex_id',
    right_on='Complex ID',
    how='left'
)

# Drop the redundant 'Complex ID' column (if desired)
merged_df.drop(columns=['Complex ID'], inplace=True)
merged_df

,station_complex_id,rmse,mae,smape,Stop Name,GTFS Latitude,GTFS Longitude
0,1,384.712868,384.712868,83.853742,Astoria-Ditmars Blvd,40.775036,-73.912034
1,10,586.580283,586.580283,199.441926,49 St,40.759901,-73.984139
2,100,252.846571,252.846571,124.411559,Hewes St,40.706870,-73.953431
3,101,152.057256,152.057256,55.905147,Marcy Av,40.708359,-73.957757
4,118,81.031381,81.031381,60.615512,3 Av,40.732849,-73.986122
...,...,...,...,...,...,...,...
407,95,2322.536609,2322.536609,161.368697,Gates Av,40.689630,-73.922270
408,96,12577.824545,12577.824545,180.450216,Kosciuszko St,40.693342,-73.928814
409,97,93.145058,93.145058,31.520772,Myrtle Av,40.697207,-73.935657
410,98,120.875473,120.875473,97.932074,Flushing Av,40.700260,-73.941126


In [13]:
station_df = pd.read_csv(r'D:\capstone_F24\data\Cleaned_data\station_data.csv')
station_df.head()

C:\Users\one1c\AppData\Local\Temp\ipykernel_2468\3692923470.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  station_df = pd.read_csv(r'D:\capstone_F24\data\Cleaned_data\station_data.csv')


,station_complex_id,station_complex,borough,latitude,longitude,Georeference,nearby_attractions
0,59,9 Av (D),Brooklyn,40.646294,-73.99432,POINT (-73.99432 40.646294),"Borough Park Manor Condominium, Boro Park Cour..."
1,404,28 St (6),Manhattan,40.743070,-73.98426,POINT (-73.98426 40.74307),"The Churchill, NYC Wheelchair Transportation, ..."
2,272,"36 St (M,R)",Queens,40.752040,-73.92878,POINT (-73.92878 40.75204),"Materials for the Arts, Carolina Power Wellnes..."
3,420,"219 St (2,5)",Bronx,40.883896,-73.86263,POINT (-73.86263 40.883896),"Bibilis Wellness Center, Agnes Haywood Playground"
4,218,"167 St (B,D)",Bronx,40.833770,-73.91844,POINT (-73.91844 40.83377),"Askari Tours, Bronx Museum of the Arts, Mullal..."


In [15]:
import folium
import branca
from folium.plugins import GroupedLayerControl

def create_combined_error_map(merged_df, station_df):
    # Create base map centered on the mean coordinates
    m = folium.Map(
        location=[
            merged_df['GTFS Latitude'].mean(),
            merged_df['GTFS Longitude'].mean()
        ],
        zoom_start=11
    )
    
    # Create colormaps for both metrics
    mae_colormap = branca.colormap.LinearColormap(
        colors=['green', 'lightgreen', 'yellow', 'orange', 'red'],
        vmin=0, vmax=1,
        caption='MAE (Normalized)'
    )
    
    smape_colormap = branca.colormap.LinearColormap(
        colors=['green', 'lightgreen', 'yellow', 'orange', 'red'],
        vmin=0, vmax=1,
        caption='SMAPE (Normalized)'
    )
    
    # Create feature groups for each metric
    mae_group = folium.FeatureGroup(name='MAE', show=True)
    smape_group = folium.FeatureGroup(name='SMAPE', show=False)
    
    # Function to format nearby attractions
    def format_attractions(attractions):
        if pd.isna(attractions):
            return "None listed"
        attractions_list = attractions.split(', ')
        if len(attractions_list) > 3:
            return ', '.join(attractions_list[:3]) + '...'
        return attractions
    
    # Add stations to MAE layer
    for _, row in merged_df.iterrows():
        # Get station information using station_complex_id
        station_info = station_df[station_df['station_complex_id'] == row['station_complex_id']]
        
        if len(station_info) > 0:
            station_info = station_info.iloc[0]
            
            tooltip_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                Borough: {station_info['borough']}<br>
                Nearby: {format_attractions(station_info['nearby_attractions'])}
            </div>
            """
            
            popup_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                MAE: {row['mae_normalized']:.3f}<br>
                SMAPE: {row['smape_normalized']:.3f}<br>
                <hr>
                Borough: {station_info['borough']}<br>
                Nearby Attractions:<br>
                {station_info['nearby_attractions']}
            </div>
            """
        else:
            # Fallback if no match found
            tooltip_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b>
            </div>
            """
            
            popup_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                MAE: {row['mae_normalized']:.3f}<br>
                SMAPE: {row['smape_normalized']:.3f}
            </div>
            """
        
        folium.CircleMarker(
            location=(row['GTFS Latitude'], row['GTFS Longitude']),
            radius=5,
            color=mae_colormap(row['mae_normalized']),
            fill=True,
            fill_color=mae_colormap(row['mae_normalized']),
            fill_opacity=0.7,
            tooltip=folium.Tooltip(tooltip_html),
            popup=folium.Popup(popup_html, max_width=300),
        ).add_to(mae_group)
    
    # Add stations to SMAPE layer
    for _, row in merged_df.iterrows():
        # Get station information using station_complex_id
        station_info = station_df[station_df['station_complex_id'] == row['station_complex_id']]
        
        if len(station_info) > 0:
            station_info = station_info.iloc[0]
            
            tooltip_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                Borough: {station_info['borough']}<br>
                Nearby: {format_attractions(station_info['nearby_attractions'])}
            </div>
            """
            
            popup_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                MAE: {row['mae_normalized']:.3f}<br>
                SMAPE: {row['smape_normalized']:.3f}<br>
                <hr>
                Borough: {station_info['borough']}<br>
                Nearby Attractions:<br>
                {station_info['nearby_attractions']}
            </div>
            """
        else:
            # Fallback if no match found
            tooltip_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b>
            </div>
            """
            
            popup_html = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>{row['Stop Name']}</b><br>
                MAE: {row['mae_normalized']:.3f}<br>
                SMAPE: {row['smape_normalized']:.3f}
            </div>
            """
        
        folium.CircleMarker(
            location=(row['GTFS Latitude'], row['GTFS Longitude']),
            radius=5,
            color=smape_colormap(row['smape_normalized']),
            fill=True,
            fill_color=smape_colormap(row['smape_normalized']),
            fill_opacity=0.7,
            tooltip=folium.Tooltip(tooltip_html),
            popup=folium.Popup(popup_html, max_width=300),
        ).add_to(smape_group)
    
    # Add feature groups to map
    mae_group.add_to(m)
    smape_group.add_to(m)
    
    # Add colormaps to map
    mae_colormap.add_to(m)
    smape_colormap.add_to(m)
    
    # Add layer control
    folium.LayerControl(collapsed=False).add_to(m)
    
    return m

# Create and save the combined map
combined_map = create_combined_error_map(merged_df, station_df)
combined_map.save("combined_error_map.html")

In [16]:
combined_map